In [ ]:
import numpy as np, matplotlib.pyplot as plt
from numpy.typing import NDArray
import matplotlib.colors as colors
import pandas as pd, seaborn as sb

In [ ]:
class MaibaumMixture:
    def __init__(self, h: float, g: float,
                 e: NDArray | list | float,
                 monomer_density: NDArray | list | float = None):
        self.h = h
        self.g = g
        self.e = np.atleast_2d(e)
        assert self.e.shape[0] == self.e.shape[1]

        if monomer_density is not None:
            monomer_density = np.atleast_1d(monomer_density)
            assert len(monomer_density) == self.m
            self.monomer_density = monomer_density
        else:
            self.monomer_density = np.ones(self.m)

    @property
    def nspecies(self):
        return len(self.e)

    @property
    def m(self):
        """Short-hand for num species."""
        return self.nspecies

    def free_energy(self, ni: NDArray | list):
        ni = np.atleast_1d(ni)
        n = np.sum(ni, axis=0)
        assert len(ni) == self.m

        with np.errstate(invalid='ignore'):
            # Composition-dependent bulk free energy (quadratic in density, so
            # treat self.e like a matrix of second virial coefficients)
            x = ni / n
            e = np.einsum('a...,ab,b...->...', x, self.e, x)

            bulk = -e * (n-1)
            surface = self.g * (n-1)**(2/3)
            stoichiometric = self.h * (n-1)**(5/3)
            ideal = np.tensordot(-np.log(self.monomer_density), ni, axes=(0, 0))
            return bulk + surface + stoichiometric + ideal

model = MaibaumMixture(1e-1, 5, 2.5)
n = np.linspace(1, 50, 1001)
G = model.free_energy([n]).reshape(-1)
plt.plot(n, np.exp(-G))
plt.xlabel('$n$')
plt.ylabel(r'$c_n / c_1 = \exp{\left(-\beta \Delta G\right)}$')
plt.ylim([0, 1])
plt.xlim([0, n[-1]])
plt.show()

Quadratic free energy density:
$$
g^\text{ex}(\vec\rho) = \frac{B_{ij} \rho^2}{2} x_i x_j + \mathcal{O}{(\rho^3)}
$$
where $x_i = n_i / n$ is the mole fraction of species $i$.

In [ ]:
eps = -0.1
B11, B12, B22 = -5 + eps, -5 - eps, -5 + eps
B = np.array([[B11, B12], [B12, B22]])
print(B)
model = MaibaumMixture(1e-1, 5, -0.5*B)

n1, n2 = np.arange(41), np.arange(41)
n1, n2 = np.meshgrid(n1, n2, indexing='ij')
G = model.free_energy([n1, n2])
n = n1 + n2

c = np.exp(-G)

fig = plt.figure(figsize=(3.375, 2.5))

mesh = plt.pcolormesh(n1, n2, c,
                      norm=colors.LogNorm(vmin=1e-2, vmax=np.max(c[n > 5])))

plt.gca().axis('equal')
fig.colorbar(mesh, extend='both', label=r'$c(\vec{n})$')
plt.xlabel('$n_1$')
plt.ylabel('$n_2$')
plt.show()


In [ ]:
np.eye(1,3,2)

In [ ]:
class BinaryAnianssonWall:
    def __init__(self, model, nmax):
        assert model.m == 2
        self.model = model

        ni = [np.arange(nmax+1) for i in range(self.m)]
        ni = np.meshgrid(*ni, indexing='ij')
        ni = [nn.reshape(-1) for nn in ni]
        n = np.sum(ni, axis=0)
        select = np.logical_and(n >= 1, n <= nmax)
        ni = [nn[select] for nn in ni]
        self.ni = np.array(ni).T
        self.n = np.sum(self.ni, axis=0)

        self.calculate_coefficients()

    @property
    def m(self):
        return self.model.m

    @property
    def num_reactions(self):
        return len(self.rate_coefficients)

    @property
    def num_elements(self):
        return len(self.ni)

    def __contains__(self, ni):
        return np.any(np.all(self.ni == ni, axis=1))

    def index(self, ni):
        return np.where(np.all(self.ni == ni, axis=1))[0]

    def calculate_coefficients(self):

        S = []
        rate_coefficients = []
        i1 = []
        i2 = []
        iprod = []

        # reaction = 0
        for k, ni in enumerate(self.ni):
            # print(f'{k:<4} {n[0]:<4} {n[1]:<4}')
            G0 = self.model.free_energy(ni)

            for i in range(self.m):
                monomer = np.eye(1,self.m,i, dtype=int).reshape(-1)
                product = ni + monomer
                if product not in self: continue

                # print(f'{reaction:<4} {ni.tolist()} <-> {product.tolist()}')
                G1 = self.model.free_energy(product)

                kb = 1.
                kf = kb * np.exp(-(G1 - G0))

                i1 += [self.index(monomer)]
                i2 += [self.index(ni)]
                iprod += [self.index(product)]

                dni = np.zeros(len(self.ni), dtype=int)
                dni[i1[-1]] -= 1
                dni[i2[-1]] -= 1
                dni[iprod[-1]] += 1

                S += [dni, -dni]
                rate_coefficients += [kf, kb]

                # reaction += 1

        self.S = np.array(S, dtype=int).T
        self.rate_coefficients = np.array(rate_coefficients)
        self.i1 = np.array(i1).reshape(-1)
        self.i2 = np.array(i2).reshape(-1)
        self.iprod = np.array(iprod).reshape(-1)

    def rhs(self, c):
        rates = np.zeros(self.num_reactions)
        rates[::2] = c[self.i1] * c[self.i2]
        rates[1::2] = c[self.iprod]
        rates *= self.rate_coefficients
        return self.S.dot(rates)

    def rate_gradient(self, c):
        kf = self.rate_coefficients[::2]
        kb = self.rate_coefficients[1::2]
        J = np.zeros((self.num_reactions, self.num_elements))
        i = 2*np.arange(self.num_reactions//2)
        J[i, self.i1] += kf * c[self.i2]
        J[i, self.i2] += kf * c[self.i1]
        J[i+1, self.iprod] = kb
        return J

    def jacobian_rhs(self, c):
        return self.S.dot(self.rate_gradient(c))

    def characteristic_rates(self, c):
        J = self.jacobian_rhs(c)
        evals = np.linalg.eigvals(J)
        order = np.argsort(np.abs(evals))
        return evals[order]

eps = -0.1
B11, B12, B22 = -5 + eps, -5 - eps, -5 + eps
B = np.array([[B11, B12], [B12, B22]])
print(B)
model = MaibaumMixture(1e-1, 5, -0.5*B)
sim = BinaryAnianssonWall(model, 40)

Geq = model.free_energy(sim.ni.T)
ceq = np.exp(-Geq)
assert np.all(np.isclose(sim.rhs(ceq), 0))


from scipy.optimize import approx_fprime
for c in [ceq, np.random.random(sim.num_elements)]:
    exact = approx_fprime(c, sim.rhs)
    assert np.allclose(sim.jacobian_rhs(c), exact, atol=1e-4, rtol=1e-4)

In [ ]:
nmax = 50
Bhom = -5

for eps in [1, 0.1, 0., -0.1]:
    B12 = Bhom + eps
    B = np.array([[Bhom, B12], [B12, Bhom]])
    model = MaibaumMixture(1e-1, 5, -0.5*B)
    sim = BinaryAnianssonWall(model, nmax)

    n1, n2 = np.arange(nmax+1), np.arange(nmax+1)
    n1, n2 = np.meshgrid(n1, n2, indexing='ij')
    G = model.free_energy([n1, n2])
    n = n1 + n2
    c = np.exp(-G)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(3.375, 3.375),
                                   height_ratios=[3, 1])

    mesh = ax1.pcolormesh(n1, n2, c,
                        norm=colors.LogNorm(vmin=1e-2, vmax=np.max(c[n > 5])))

    ax1.set_xlim([0, nmax])
    ax1.axis('equal')
    fig.colorbar(mesh, extend='both', label=r'$c(\vec{n})$')
    ax1.set_xlabel('$n_1$')
    ax1.set_ylabel('$n_2$')

    Geq = model.free_energy(sim.ni.T)
    ceq = np.exp(-Geq)
    evals = np.log10(-sim.characteristic_rates(ceq)[2:])
    evals = pd.DataFrame(evals)
    sb.boxplot(ax=ax2, data=evals, showfliers=False, orient='h',
               width=0.95, linewidth=0.5, boxprops={'facecolor':'None'})
    sb.swarmplot(ax=ax2, data=evals, orient='h',
                 edgecolor=['k']*len(evals), linewidth=0.25, size=2.5)

    import matplotlib.ticker as ticker
    ax2.xaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: rf'$10^{{{y:g}}}$'))
    ax2.spines[['right', 'top', 'left']].set_visible(False)
    ax2.set_yticks([])
    ax2.set_xlabel(r'$|\lambda_i|$')

    matrix = rf'$\mathbf{{B}} = \begin{{pmatrix}} {Bhom:.1f} & {B12:.1f} \\ {B12:.1f} & {Bhom:.1f} \end{{pmatrix}}$'
    text = ax1.text(nmax, nmax, matrix, c='white', #transform=ax1.transAxes,
                    ha='right', va='top', fontsize=8)
    text.set_in_layout(False)

plt.show()